In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
%matplotlib inline 
from __future__ import division
from sympy import Symbol, init_printing
from sympy.solvers import solve
from IPython.display import display
init_printing() 

In [10]:
#Pick a random value of z and generate training points (x1,x2, y) such that x1,x2 & y are conditionally independent given z
pz = np.random.random()
x1_z0 = np.random.random()
x1_z1 = np.random.random()
x2_z0 = np.random.random()
x2_z1 = np.random.random()
y_z0 = np.random.random()
y_z1 = np.random.random()

px1 = pz*x1_z0+(1-pz)*x1_z1
px2 = pz*x2_z0+(1-pz)*x2_z1
py = pz*y_z0+(1-pz)*y_z1

n        = 1 #Bernoulli random variable
nsamples = 10000
x1_train  = np.random.binomial(n, 1-px1, size=(nsamples,))
x2_train  = np.random.binomial(n, 1-px2, size=(nsamples,))
y_train  = np.random.binomial(n, 1-py, size=(nsamples,))

#generate test data such that p(y|z) and p(x2|z) is the same but p(x1|z) is different
new_x1_z0 = np.random.random()
new_x1_z1 = np.random.random()

new_px1 = pz*new_x1_z0+(1-pz)*new_x1_z1
x1_test  = np.random.binomial(n, 1-new_px1, size=(nsamples,))
x2_test  = np.random.binomial(n, 1-px2, size=(nsamples,))
y_test  = np.random.binomial(n, 1-py, size=(nsamples,))

#pick a random z, find p(x|z), p(y|z)
temp_z = np.random.random()

#estimates for p(x1), p(x2),p(y), p(x1,x2), p(x1,y),p(x2,y) from data
cal_x1 = sum(x1_train ==0)/nsamples
cal_x2 = sum(x2_train ==0)/nsamples
cal_y = sum(y_train ==0)/nsamples

cal_x1x2 =  sum([u==0 and v==0 for u,v in zip(x1_train,x2_train)])/nsamples
cal_x1y =  sum([u==0 and v==0 for u,v in zip(x1_train,y_train)])/nsamples
cal_x2y =  sum([u==0 and v==0 for u,v in zip(x2_train,y_train)])/nsamples


In [6]:
#Solving for the conditional probabilities p(x1 =0|z=0),p(x1 =0|z=1), p(x2=0|z=0),p(x2=0|z=1), p(y=0|z=0),p(y=0|z=1)
x1 = Symbol('x1')
x2 = Symbol('x2')
y  = Symbol('y')

p12 = Symbol('p12')
p1y = Symbol('p1y')
p2y = Symbol('p2y')

a = Symbol('a')
b = Symbol('b')
c = Symbol('c')
d = Symbol('d')
e = Symbol('e')
f = Symbol('f')

z0 = Symbol('z0')

sols = solve([
    z0 * a + (1 - z0) * b - x1,
    z0 * c + (1 - z0) * d - x2,
    z0 * e + (1 - z0) * f - y,

    z0 * a * c + (1 - z0) * b * d - p12,
    z0 * a * e + (1 - z0) * b * f - p1y,
    z0 * c * e + (1 - z0) * d * f - p2y
], a, b, c, d, e, f, dict=True)

def compute_sols(x1_, x2_, y_, p12_, p1y_, p2y_, z0_):
    def evaluate(s):
        return s.evalf(subs={x1: x1_, x2: x2_, y: y_, p12: p12_, p1y: p1y_, p2y: p2y_, z0: z0_})
    
    return [dict([(k, evaluate(s)) for k, s in sols[i].items()]) for i in range(2)]


In [11]:
print(compute_sols(cal_x1, cal_x2, cal_y, cal_x1x2, cal_x1y, cal_x2y, temp_z))

[{f: 0.4507 + 0.0573576708935612*I, c: 0.1347 - 0.0454915612742027*I, d: 0.1347 + 0.0545915413981772*I, a: 0.3676 - 0.0222693840265991*I, e: 0.4507 - 0.0477965987618572*I, b: 0.3676 + 0.026724077300231*I}, {f: 0.4507 - 0.0573576708935612*I, c: 0.1347 + 0.0454915612742027*I, d: 0.1347 - 0.0545915413981772*I, a: 0.3676 + 0.0222693840265991*I, e: 0.4507 + 0.0477965987618572*I, b: 0.3676 - 0.026724077300231*I}]


In [ ]:
'''
p_yx = (pz*y_z0*x_z0 + (1-pz)*y_z1*x_z1)/px
new_p_yx = (pz*y_z0*x_z0 + (1-pz)*y_z1*new_x_z1)/new_px


# cal_x = pz* a+(1-pz)*b
# cal_y = pz*c + (1-pz)*d
# p_yx * cal_x = pz* ac + (1-pz)* bd 

pred_y_z0=
pred_y_z1=

pred_x_z0=
pred_x_z1=

#see how p(y|x) compares to the true underlying distribution

pred_p_yx = (pz*pred_y_z0*pred_x_z0 + (1-pz)*pred_y_z1*pred_x_z1)/cal_x

new_pred_p_yx = (pz*pred_y_z0*new_pred_x_z0 + (1-pz)*pred_y_z1*new_pred_x_z1)/cal_x

#Given p_test(x), find new p_test(x|z), and find p(y|x) for test

cal_x_test = sum(x_train ==0)/nsamples
'''